<a href="https://colab.research.google.com/github/TDStriker/Projects-in-ML/blob/main/ML_Project_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install datasets

In [1]:
!pip install transformers[torch]

In [2]:
!pip install accelerate -U

In [1]:
from transformers import pipeline
import pandas
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk

NLP

The dataset is a collection of CNN news articles

https://data.world/opensnippets/cnn-news-dataset

In [4]:
!unzip cnn_data.zip

Archive:  cnn_data.zip
replace cnn_data copy.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
articles = pandas.read_json("cnn_data copy.json")

In [3]:
articles

,title,url,published_at,last_modified_at,author,short_description,header_image,raw_content,content,crawled_at,_id,source
0,Fleetwood and Hoey lead big names in Dunhill L...,https://edition.cnn.com/2011/09/30/sport/golf/...,2011-09-30 19:30:12+00:00,2011-09-30 20:19:30+00:00,By,Unheralded pair Tommy Fleetwood of England and...,https://cdn.cnn.com/cnnnext/dam/assets/1109300...,"<div class=""l-container""><div class=""el__leafm...",Fleetwood and Hoey lead big names in Dunhill L...,2021-07-23 00:15:23,0d641f2c-1b43-5651-adb5-18ccea4df489,edition.cnn.com
1,Senna to replace Heidfeld in Belgium,https://edition.cnn.com/2011/08/24/sport/motor...,2011-08-24 17:54:07+00:00,2011-08-26 16:49:15+00:00,By,Brazilian driver Bruno Senna will replace Nick...,https://cdn.cnn.com/cnnnext/dam/assets/1108240...,"<div class=""l-container""><div class=""el__leafm...",Senna to replace Heidfeld in BelgiumBy Updated...,2021-07-23 00:15:23,2419cdcf-50a7-5b04-a9fc-fd7813731fe9,edition.cnn.com
2,Zvonareva beats Kvitova to reach Tokyo final,https://edition.cnn.com/2011/09/30/sport/tenni...,2011-09-30 14:59:26+00:00,2011-09-30 20:22:34+00:00,By,Russian Vera Zvonareva came from 5-1 down in t...,https://cdn.cnn.com/cnnnext/dam/assets/1109300...,"<div class=""l-container""><div class=""el__leafm...",Zvonareva beats Kvitova to reach Tokyo finalBy...,2021-07-23 00:15:23,fc9c8863-007f-540f-9f6f-4981f2ae6cb6,edition.cnn.com
3,Country profile: Macedonia,https://edition.cnn.com/2011/09/30/world/europ...,2011-09-30 04:34:03+00:00,2011-09-30 04:34:03+00:00,By Catriona Davies and Eoghan Macguire for CNN,Macedonia is a small landlocked country border...,https://cdn.cnn.com/cnnnext/dam/assets/1109291...,"<div class=""l-container""><div class=""el__leafm...",Country profile: MacedoniaBy Catriona Davies a...,2021-07-23 00:15:23,f7625158-e643-5859-819c-6736f7e40d31,edition.cnn.com
4,Manchester City ban talk of Tevez,https://edition.cnn.com/2011/09/30/sport/footb...,2011-09-30 16:54:25+00:00,2011-09-30 16:55:10+00:00,By,Manchester City banned journalists from asking...,https://cdn.cnn.com/cnnnext/dam/assets/1109300...,"<div class=""l-container""><div class=""el__leafm...",Manchester City ban talk of TevezBy Updated 16...,2021-07-23 00:15:24,91e56668-cad7-5654-ad7f-770c92ac443e,edition.cnn.com
...,...,...,...,...,...,...,...,...,...,...,...,...
3495,Kremlin critic Lebedev: Why I lashed out on TV...,https://edition.cnn.com/2012/10/01/world/europ...,2012-10-01 11:47:24+00:00,2012-10-01 12:38:45+00:00,"By Phil Black and Peter Wilkinson, CNN",Russian billionaire businessman and outspoken ...,https://cdn.cnn.com/cnnnext/dam/assets/1209300...,"<div class=""l-container""><div class=""el__leafm...",Kremlin critic Lebedev: Why I lashed out on TV...,2021-07-23 00:23:05,946e3153-3697-558d-80e7-1c0587174d88,edition.cnn.com
3496,Never forget: Football remembers,https://edition.cnn.com/2012/11/30/sport/footb...,2012-11-30 20:55:24+00:00,2012-12-01 18:46:18+00:00,"By James Masters, CNN",The Premier League's 'Christmas Truce' tournam...,https://cdn.cnn.com/cnnnext/dam/assets/1211300...,"<div class=""l-container""><div class=""el__leafm...",Never forget: Football remembersBy James Maste...,2021-07-23 00:23:05,855f08d9-9872-51aa-b93b-a576b2d4b46b,edition.cnn.com
3497,Schumacher coy over Sauber switch,https://edition.cnn.com/2012/10/01/sport/motor...,2012-10-01 15:51:41+00:00,2012-10-01 16:02:52+00:00,By,Michael Schumacher is keeping all his options ...,https://cdn.cnn.com/cnnnext/dam/assets/1210010...,"<div class=""l-container""><div class=""el__leafm...",Schumacher coy over Sauber switchBy Updated 16...,2021-07-23 00:23:05,437298b0-aa5d-57a9-9898-49cac6a7ba70,edition.cnn.com
3498,It's official: Vettel is three-time champion a...,https://edition.cnn.com/2012/11/30/sport/motor...,2012-11-30 12:21:35+00:00,2012-11-30 12:21:35+00:00,By,It's official: Sebastian Vettel is a three-tim...,https://cdn.cnn.com/cnnnext/dam/assets/1211281...,"<div class=""l-container""><div class=""el__leafm...",It's offic

In [4]:
articles = articles.drop(columns=["title","published_at","last_modified_at","header_image","raw_content","crawled_at","_id","source","url","author"])

In [5]:
articles

,short_description,content
0,Unheralded pair Tommy Fleetwood of England and...,Fleetwood and Hoey lead big names in Dunhill L...
1,Brazilian driver Bruno Senna will replace Nick...,Senna to replace Heidfeld in BelgiumBy Updated...
2,Russian Vera Zvonareva came from 5-1 down in t...,Zvonareva beats Kvitova to reach Tokyo finalBy...
3,Macedonia is a small landlocked country border...,Country profile: MacedoniaBy Catriona Davies a...
4,Manchester City banned journalists from asking...,Manchester City ban talk of TevezBy Updated 16...
...,...,...
3495,Russian billionaire businessman and outspoken ...,Kremlin critic Lebedev: Why I lashed out on TV...
3496,The Premier League's 'Christmas Truce' tournam...,Never forget: Football remembersBy James Maste...
3497,Michael Schumacher is keeping all his options ...,Schumacher coy over Sauber switchBy Updated 16...
3498,It's official: Sebastian Vettel is a three-tim...,It's official: Vettel is three-time champion a...


In [6]:
part = int(len(articles)*9/10)
test = articles.iloc[part:]
train = articles.iloc[:part]

In [7]:
#Untrained summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
print(summarizer(articles["content"].iloc[1]))

[{'summary_text': 'Bruno Senna will replace Nick Heidfeld for Renault at the Belgian Grand Prix. Senna is the nephew of three-time world champion Ayrton Senna. The Brazilian has not raced since the season-ending Abu Dhabi Grand Prix in November 2010. The 34-year-old has picked up 34 points so far this season.'}]


In [1]:
import datasets
from datasets import load_dataset

dataset = load_dataset("json", data_files="cnn_data copy.json",split="train[:90%]")
valset = load_dataset("json", data_files="cnn_data copy.json",split="train[90%:]")

In [2]:
dataset

Dataset({
    features: ['last_modified_at', 'raw_content', 'source', 'url', 'content', 'author', 'crawled_at', 'title', 'published_at', 'header_image', 'short_description', '_id'],
    num_rows: 3150
})

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")


def tokenize_function(examples):
    model_inputs = tokenizer(examples["content"], padding='max_length', truncation=True)

    with tokenizer.as_target_tokenizer():
      targets = tokenizer(examples['short_description'], padding='max_length', truncation=True)

    model_inputs['labels'] = targets['input_ids']

    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_valsets = valset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
print(tokenized_datasets)

Dataset({
    features: ['last_modified_at', 'raw_content', 'source', 'url', 'content', 'author', 'crawled_at', 'title', 'published_at', 'header_image', 'short_description', '_id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3150
})


In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(["last_modified_at","raw_content","source","url","content","author","crawled_at","title","published_at","header_image","_id","short_description"])
tokenized_valsets = tokenized_valsets.remove_columns(["last_modified_at","raw_content","source","url","content","author","crawled_at","title","published_at","header_image","_id","short_description"])

In [6]:
print(tokenized_datasets)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3150
})


In [7]:
# tokenized_datasets = tokenized_datasets.rename_column("short_description", "labels")
# tokenized_valsets = tokenized_valsets.rename_column("short_description", "labels")

In [8]:
# print(tokenized_datasets)

In [9]:
tokenized_datasets.set_format("torch")
tokenized_valsets.set_format("torch")

In [10]:
train_dataset = tokenized_datasets.shuffle(seed=42)
test_dataset = tokenized_valsets.shuffle(seed=42)

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(test_dataset, batch_size=8)

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=5)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-3)

In [14]:
num_epochs = 7

In [15]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,),

In [ ]:
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        items = batch.items()
        batch = {k: v for k, v in items}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
model.eval()
for batch in eval_dataloader:
    items = batch.items()
    batch = {k: v for k, v in items}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    rouge.add_batch(predictions=predictions, references=batch["labels"])
    bleu.add_batch(predictions=predictions, references=batch["labels"])

print(rouge.compute())
print(bleu.compute())

The hyperparameters used were crucial for the impact of the training. The original learning rate being used was too small so I had to increase it in order to get the desired effects. Increasing batch size also helped the training process. Increasing num_epochs helped improve accuracy, but when it was increased too high it caused the program to crash from using too much memory.

The choice of LLM definitely impacts results. BART base is a smaller model compared to ones like GPT-3, which definitely reduces the quality of the results compared to such models. BART does happen to excel in summarization however, sowhile overall the model might not perform as well it still is able to produce exceptional results.

RL

Task 2: Driving a car to a destination can be formulated as an MDP. The states would be forks in the road, the actions would be the different paths you can take, and the transition model would be that whichever path you choose deterministically leads to the next state. The reward function would have negative rewards for gas consumption or time spent to incentivise quicker and more fuel efficient routes. There would also be a large positive reward at the destination.

Task 3:

In the field of healthcare, reinforcement learning works well for deciding treatment types and scheduling. This works well in the structure of RL as it can be modelled based on patient condition and treatment history, with treatment types and dosages serving as different types of action for the agent to chose from. Backed with knowledge from a deep RL network it may be able to efficiently determine treatments for patients.

The article linked below uses Deep Reinforcement Learning to produce personalized treatment recommendation for patients. They propose using actor-critic methods to create a personalized ranking of different treatment options. The model starts by using an MDP to rank all of the candidate drugs. Each ranking is represented by a discrete time step in the MDP. At each time step the agent receives information on the previously ranked drugs and the ones that remain as candidates. The agent then uses its policy to select one of the remaining drugs to choose for the current ranking before moving to the next time step.

The actor network consists of a MF layer followed up by an embedding and stacking layer that reformulates the data before sending it to the both a Cross Network and a Deep Network. The Cross Network is intended to learn high order interactions between cells and drugs, while the Deep Network attempts to gain generalized knowledge to help the model. The model also implements a GRU in an attempt to learn hidden preferences of the patients over time. The output from the GRU is then sent to a Concatenation Layer alongside the Cross and Deep Network outputs, where the model then calculates the final policy.

The critic consists of several cross and deep layers followed by a feed foward layer. It takes the actor's policy as input and outputs a valuation to guide the model and its training.

For this research they focused on ranking the top k drugs rather than trying to rank every single drug available.

https://onlinelibrary.wiley.com/doi/epdf/10.1002/sim.9491